In [1]:

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
img = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)
kernel = [[0, 1, 1, 1, 0],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [0, 1, 1, 1, 0]]
kernel = np.asarray(kernel,dtype=np.int32)

In [4]:
def dilation(img, kernel):
    res_img = np.zeros(shape=img.shape,dtype=np.uint8)
    width = kernel.shape[0]
    height = kernel.shape[1]
    center = (height//2,width//2)
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            res_img[row,col] = img[row,col]
            # kernel
            max_val = 0
            for krow in range(kernel.shape[0]):
                for kcol in range(kernel.shape[1]):
                    if kernel[krow,kcol] != 1:
                        continue
                    x = row + krow-center[0]
                    y = col + kcol-center[1]
                    if x < 0 or y < 0 or x >= img.shape[0] or y >= img.shape[1]:
                        continue
                    max_val = img[x,y] if img[x,y] > max_val else max_val
            res_img[row,col] = max_val
    return res_img
                        
                
dilation_img = dilation(img,kernel)
cv2.imshow("Dilation",dilation_img)
cv2.waitKey()
cv2.imwrite("Dilation.png",dilation_img)

True

In [5]:
def erosion(img, kernel):
    res_img = np.ones(shape=img.shape,dtype=np.uint8)
    width = kernel.shape[0]
    height = kernel.shape[1]
    center = (height//2,width//2)
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            res_img[row,col] = img[row,col]
            # kernel
            min_val = 255
            for krow in range(kernel.shape[0]):
                for kcol in range(kernel.shape[1]):
                    if kernel[krow,kcol] != 1:
                        continue
                    x = row + krow-center[0]
                    y = col + kcol-center[1]
                    if x < 0 or y < 0 or x >= img.shape[0] or y >= img.shape[1]:
                        min_val = 0
                        break
                    min_val = img[x,y] if img[x,y] < min_val else min_val
            res_img[row,col] = min_val
                    
    return res_img
                        
                
erosion_img = erosion(img,kernel)
cv2.imshow("Erosion",erosion_img)
cv2.waitKey()
cv2.imwrite("Erosion.png",erosion_img)

True

In [6]:
opening_img = dilation(erosion_img,kernel)
cv2.imshow("Opening",opening_img)
cv2.waitKey()
cv2.imwrite("Opening.png",opening_img)
closing_img = erosion(dilation_img,kernel)
cv2.imshow("Closing",closing_img)
cv2.waitKey()
cv2.imwrite("Closing.png",closing_img)

True